In [1]:
import pyomo.environ as pyo
import numpy as np


In [2]:
lmda = 21.97 #cent/kWh retail electricity price
pl = [8,8,10,10,10,16,22,24,26,32,30,28,22,18,16,16,20,24,28,34,38,30,22,12] #load values

# fuel cost parameters
c2 = np.array([1.2,1.12])*0.001
c1 = np.array([0.128,0.532])
c  = np.array([2.12,12.8])*0.0001

# min max power values
pmin = np.array([0,0])
pmax = np.array([20,40])

In [11]:
# Dont properly understand Difference between Concrete and Abstract model
model = pyo.ConcreteModel()

#Hour Set
model.H = pyo.RangeSet(0,23)

#Generator Set
model.G = pyo.RangeSet(0,1)

#Variables
model.pn = pyo.Var(model.H) #Net power needed from external network dependent on Hourly Set
model.pg = pyo.Var(model.H,model.G,within=pyo.NonNegativeReals) #Power Generation of each Generator per Hour
#model.u = pyo.Var(model.H,model.G, within=pyo.Binary) # Unit commitment Variable for each Generator and Hour
model.x = pyo.Var(model.H,model.G,within=pyo.NonNegativeReals) # continuous Variable to substitue binary


#Objective Function: First Part is Net power cost with DisCo, Second part is fuel costs


model.OBJ = pyo.Objective(expr= 
                          sum(lmda*model.pn[h] for h in model.H)
                         +sum(((c2[g]*model.pg[h,g])**2 + c1[g]*model.pg[h,g] + c[g])*model.x[h,g] for h in model.H for g in model.G))



#Load Constraint
def loadc(model,H):
    return sum(model.pg[H,g] for g in model.G) +model.pn[H] == pl[H]
model.loadc = pyo.Constraint(model.H, rule = loadc)

#Min-Constraint
def minc(model,H,G):
    return model.x[H,G]*pmin[G] <= model.pg[H,G] 
model.minc = pyo.Constraint(model.H,model.G, rule = minc)

#Max-Constraint
def maxc(model,H,G):
    return model.x[H,G]*pmax[G] >= model.pg[H,G] 
model.maxc = pyo.Constraint(model.H,model.G, rule = maxc)

#Continuous-Binary constraint
def cbc(model,H,G):
    return (model.x[H,G]**2)-1 == 0
model.cbc = pyo.Constraint(model.H,model.G, rule = cbc)
    
    
    

opt = pyo.SolverFactory('gurobi')
succes = opt.solve(model)
succes.write()

RuntimeError: Cannot write legal LP file.  Objective 'OBJ' has nonlinear terms that are not quadratic.